### Get results from Wandb

In [ ]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_finetune")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

### Read runs

In [30]:
data = pd.read_csv('project.csv', index_col=0)
data

,summary,config,name
0,{'_wandb': {'runtime': 1}},"{'exp_name': 'curl_impala', 'overrides': [], '...",quiet-oath-2825
1,"{'_runtime': 22591, 'eval_mean_traj_rewards': ...","{'env': {'game': 'ms_pacman', 'type': 'atari',...",resilient-pyramid-2821
2,"{'train_mean_traj_game_scores': 7680, '_step':...","{'env': {'game': 'road_runner', 'type': 'atari...",feasible-microwave-2820
3,"{'_step': 100000, '_runtime': 22576, '_timesta...","{'env': {'game': 'private_eye', 'type': 'atari...",sweet-wind-2818
4,"{'_wandb': {'runtime': 22401}, '_runtime': 224...","{'env': {'game': 'kung_fu_master', 'type': 'at...",whole-valley-2817
...,...,...,...
2415,"{'_step': 1000, '_runtime': 14, '_timestamp': ...","{'env': {'game': 'breakout', 'type': 'atari', ...",sandy-moon-11
2416,"{'_step': 1000, '_runtime': 14, '_timestamp': ...","{'env': {'game': 'alien', 'type': 'atari', 'ho...",twilight-darkness-16
2417,"{'train_mean_traj_rewards': 7, 'train_mean_tra...","{'env': {'game': 'qbert', 'type': 'atari', 'ho...",faithful-dust-4
2418,"{'train_mean_traj_game_scores': 2.5, '_step': ...","{'env': {'game': 'breakout', 'type': 'atari', ...",smooth-smoke-13


### Filter based on conditions

In [36]:
configs = data['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    exp_name = cfg['exp_name']
    
    # condition
    if exp_name == 'byol_impala':
        indexs.append(idx)

In [37]:
data = data.iloc[indexs]
data

,summary,config,name
21,{'_wandb': {'runtime': 1}},"{'exp_name': 'byol_impala', 'overrides': [], '...",dainty-silence-2804
22,"{'eval_mean_traj_rewards': 37.70000076293945, ...","{'env': {'game': 'seaquest', 'type': 'atari', ...",sweet-aardvark-2802
23,{'train_mean_traj_rewards': 56.099998474121094...,"{'env': {'game': 'up_n_down', 'type': 'atari',...",fluent-glade-2803
38,"{'eval_mean_traj_rewards': 22.799999237060547,...","{'env': {'game': 'road_runner', 'type': 'atari...",clear-eon-2780
39,"{'_runtime': 28045, '_timestamp': 1660321112, ...","{'env': {'game': 'private_eye', 'type': 'atari...",exalted-grass-2785
...,...,...,...
2059,"{'train_loss': 0.6490869304472581, 'train_mean...","{'env': {'game': 'alien', 'type': 'atari', 'ho...",fresh-spaceship-761
2060,"{'eval_mean_traj_rewards': -6.099999904632568,...","{'env': {'game': 'boxing', 'type': 'atari', 'h...",floral-snow-766
2061,"{'_wandb': {'runtime': 28053}, '_runtime': 280...","{'env': {'game': 'crazy_climber', 'type': 'ata...",clean-firebrand-759
2062,"{'train_loss': 0.3515591138470918, 'train_mean...","{'env': {'game': 'battle_zone', 'type': 'atari...",glowing-armadillo-764


### Generate json file

In [44]:
def get_results_dict(data):
    results = {}
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results[idx] = {'0':0, '1':game, '2':score, '3': 0}

    return results
    
results

qbert
up_n_down
seaquest
pong


{1: {'0': 0, '1': 'seaquest', '2': 754, '3': 0},
 2: {'0': 0, '1': 'up_n_down', '2': 969, '3': 0},
 3: {'0': 0, '1': 'road_runner', '2': 4300, '3': 0},
 4: {'0': 0, '1': 'private_eye', '2': 100, '3': 0},
 5: {'0': 0, '1': 'pong', '2': -15.100000381469728, '3': 0},
 6: {'0': 0, '1': 'frostbite', '2': 235, '3': 0},
 7: {'0': 0, '1': 'jamesbond', '2': 190, '3': 0},
 8: {'0': 0, '1': 'kangaroo', '2': 320, '3': 0},
 9: {'0': 0, '1': 'ms_pacman', '2': 674, '3': 0},
 10: {'0': 0, '1': 'hero', '2': 7415.5, '3': 0},
 11: {'0': 0, '1': 'qbert', '2': 1857.5, '3': 0},
 12: {'0': 0, '1': 'gopher', '2': 98, '3': 0},
 13: {'0': 0, '1': 'krull', '2': 2512, '3': 0},
 14: {'0': 0, '1': 'kung_fu_master', '2': 640, '3': 0},
 15: {'0': 0, '1': 'chopper_command', '2': 640, '3': 0},
 16: {'0': 0, '1': 'assault', '2': 581.2000122070312, '3': 0},
 17: {'0': 0, '1': 'asterix', '2': 630, '3': 0},
 18: {'0': 0, '1': 'bank_heist', '2': 80, '3': 0},
 19: {'0': 0, '1': 'breakout', '2': 9.199999809265137, '3': 0},
 2